In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from skimage.color import rgb2gray
from skimage.transform import rotate
from skimage.transform import (hough_line, hough_line_peaks)
from scipy.stats import mode
from skimage import io
from skimage.filters import threshold_otsu, sobel
from matplotlib import cm


def binarize_image(rgb_image):
    image = rgb2gray(rgb_image)
    threshold = threshold_otsu(image)
    binary_image = image < threshold

    return binary_image


def find_edges(binary_image):
    image_edges = sobel(binary_image)

    plt.imshow(binary_image, cmap='gray')
    plt.axis('off')
    plt.title('Binary Image Edges')
    plt.savefig('/content/drive/My Drive/binary_image.png')

    return image_edges


def find_tilt_angle(image_edges):
    h, theta, d = hough_line(image_edges)
    accum, angles, dists = hough_line_peaks(h, theta, d)
    angle = np.rad2deg(mode(angles)[0][0])

    if angle < 0:
        r_angle = angle + 90
    else:
        r_angle = angle - 90

    # Plot Image and Lines
    fig, ax = plt.subplots()

    ax.imshow(image_edges, cmap='gray')

    origin = np.array((0, image_edges.shape[1]))

    for _, angle, dist in zip(*hough_line_peaks(h, theta, d)):
        y0, y1 = (dist - origin * np.cos(angle)) / np.sin(angle)
        ax.plot(origin, (y0, y1), '-r')

    ax.set_xlim(origin)
    ax.set_ylim((image_edges.shape[0], 0))
    ax.set_axis_off()
    ax.set_title('Detected lines')

    plt.savefig('/content/drive/My Drive/hough_lines.png')
    plt.show()

    return r_angle


def rotate_image(rgb_image, angle):
    fixed_image = rotate(rgb_image, angle)

    plt.imshow(fixed_image)
    plt.axis('off')
    plt.title('Fixed Image')
    plt.savefig('/content/drive/My Drive/fixed_image.png')
    plt.show()

    return fixed_image


def deskew_image(img_path):
    image = io.imread(img_path)
    binary_image = binarize_image(image)
    image_edges = find_edges(binary_image)
    angle = find_tilt_angle(image_edges)
    rotate_image(io.imread(img_path), angle)


deskew_image("/content/drive/MyDrive/IPDA dataset/ipda22_datasets/3_deskewing/fake_id_10__front.png")


ValueError: ignored